<a href="https://colab.research.google.com/github/Shubhamkumar-op/Deep_Learning/blob/main/NPL_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-10c0b7b5-fa61-a9da-2554-d5592d59b407)


In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-04-30 23:25:47--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-04-30 23:25:48 (105 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import unzip_data,create_tensorboard_callback,plot_loss_curves,compare_historys 

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip 

--2023-04-30 23:25:51--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.12.128, 172.217.194.128, 172.253.118.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.12.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K   753KB/s    in 0.8s    

2023-04-30 23:25:53 (753 KB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
unzip_data("nlp_getting_started.zip")

Become one with the data

In [6]:
import pandas as pd

In [7]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [8]:
train_df.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [9]:
train_df_shuffle = train_df.sample(frac=1, random_state=42)
train_df_shuffle.head(10)

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
5559,7934,rainstorm,NaN,@Calum5SOS you look like you got caught in a r...,0
1765,2538,collision,NaN,my favorite lady came to our volunteer meeting...,1
1817,2611,crashed,NaN,@brianroemmele UX fail of EMV - people want to...,1
6810,9756,tragedy,"Los Angeles, CA",Can't find my ariana grande shirt this is a f...,0
4398,6254,hijacking,"Athens,Greece",The Murderous Story Of AmericaÛªs First Hijac...,1


In [10]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [11]:
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffle[['text','target']][random_index:random_index+5].itertuples():
  _,text,target = row
  print(f"Target : {target}","(real diaster)" if target>0 else "(not real diaster)")
  print(f"Text\n{text}\n") 

Target : 0 (not real diaster)
Text
On this day in 1998 Appetite For Destruction goes to #1 on the Billboard Album Charts and stay on for 57 weeks

Target : 1 (real diaster)
Text
Memphis always floods

Target : 0 (not real diaster)
Text
@_wildfire__ ???? Bruh that's the lady from Mulan!!

Target : 0 (not real diaster)
Text
I liked a @YouTube video from @gassymexican http://t.co/lPgFqnpjd3 WHEN REALITIES COLLIDE! (Life Is Strange Hilarious Glitch)

Target : 0 (not real diaster)
Text
In @edfringe? We highly recommend @M00NF00L #Titania @Summerhallery A whirlwind reimagining /Shakespeare's Midsummer https://t.co/iIAIGZkbnJ



In [12]:
from sklearn.model_selection import train_test_split  

In [13]:
train_sentence , val_sentence , train_labels , val_labels = train_test_split(train_df_shuffle["text"].to_numpy(),
                                                                            train_df_shuffle["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [14]:
len(train_sentence),len(val_sentence),len(train_labels),len(val_labels)

(6851, 762, 6851, 762)

In [15]:
 #converting text into numbers
 import tensorflow as tf
 from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [16]:
text_vector = TextVectorization(max_tokens=10000,
                                standardize="lower_and_strip_punctuation",
                                split="whitespace",
                                ngrams=None,
                                output_mode="int",
                                output_sequence_length=None,
                                pad_to_max_tokens=True)

In [17]:
max_vocab_length = 10000
max_length = 15
text_vector = TextVectorization(max_tokens=10000,
                                output_mode="int",
                                output_sequence_length=max_length)

In [18]:
#Fit the text vectorization to the training text
text_vector.adapt(train_sentence)

In [19]:
sample_sentence = "Advice from Noah: Dont go running in a thunderstorm"
text_vector([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[6258,   20,    1,   63,  112,  876,    4,    3,  283,    0,    0,
           0,    0,    0,    0]])>

In [20]:
random_sentence = random.choice(train_sentence)
print(f"Original text:\n {random_sentence}\
                       \n\nVectorized version")
text_vector([random_sentence])

Original text:
 Fears over missing migrants in Med: Rescuers search for survivors after a boat carrying as many as 600 migrantsÛ_ http://t.co/v9ftYB30EI                       

Vectorized version


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1526,   60,  373,  330,    4, 1124,  341, 1259,   10,  362,   43,
           3,  405, 1315,   26]])>

Creating embedding using embedding layers

In [21]:
from tensorflow.keras import layers

In [22]:
embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             input_length=max_length)
embedding

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [24]:
model_0 = Pipeline([
    ("tfidf",TfidfVectorizer()),
    ("clf",MultinomialNB())
])
model_0.fit(train_sentence,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [25]:
baseline_score = model_0.score(val_sentence,val_labels)

In [26]:
baseline_score*100

79.26509186351706

In [27]:
baseline_preds = model_0.predict(val_sentence)

In [28]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

def calculate_result(y_true,y_pred):
    model_accuracy = accuracy_score(y_true,y_pred)*100
    model_precision, model_recall, model_f1,_ = precision_recall_fscore_support(y_true,y_pred,average="weighted")
    model_result = {"accuracy":model_accuracy,
                    "precision":model_precision,
                    "recall":model_recall,
                    "f1":model_f1}
    return model_result

In [29]:
baseline_result = calculate_result(y_true=val_labels,
                                   y_pred=baseline_preds)
baseline_result

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [30]:
# model_1 LSTM
# Long short term memory

from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype="string")
x = text_vector(inputs)
x = embedding(x)
# x = layers.LSTM(64,return_sequences=True)(x)
x = layers.LSTM(64)(x)
# x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs,outputs,name="model_1_LSTM")

In [31]:
model_1.summary()

Model: "model_1_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [32]:
model_1.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [33]:
model_1_history = model_1.fit(train_sentence,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentence,val_labels)) 

Epoch 1/5
215/215 [==============================] - 31s 98ms/step - loss: 0.5139 - accuracy: 0.7415 - val_loss: 0.4673 - val_accuracy: 0.7822
Epoch 2/5
215/215 [==============================] - 3s 16ms/step - loss: 0.3184 - accuracy: 0.8701 - val_loss: 0.4631 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.2152 - accuracy: 0.9215 - val_loss: 0.5597 - val_accuracy: 0.7690
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1439 - accuracy: 0.9467 - val_loss: 0.7291 - val_accuracy: 0.7677
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1011 - accuracy: 0.9615 - val_loss: 0.8578 - val_accuracy: 0.7480


In [34]:
model_1_pred_prob = model_1.predict(val_sentence)
model_1_pred = tf.squeeze(tf.round(model_1_pred_prob))
model_1_pred

24/24 [==============================] - 0s 3ms/step


<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0.,
       1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1.,
       0., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0.,
       0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1.,
       1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1.,
       0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0

In [35]:
model_1_result = calculate_result(y_true=val_labels,
                                  y_pred=model_1_pred)
model_1_result


{'accuracy': 74.80314960629921,
 'precision': 0.7484909900879931,
 'recall': 0.7480314960629921,
 'f1': 0.7461552553767129}

In [36]:
tf.random.set_seed(42)
from tensorflow.keras import layers
model_2_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_4")

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vector(inputs)
x = model_2_embedding(x)

x = layers.Bidirectional(layers.GRU(64))(x) 
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_4_Bidirectional")

In [39]:
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [37]:
model_2.summary()

Model: "model_4_Bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_4 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              74496     
 l)                                                              
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,354,625
Trainable params: 1,3

In [41]:
model_2_history = model_2.fit(train_sentence,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentence, val_labels))

Epoch 1/5
215/215 [==============================] - 17s 57ms/step - loss: 0.5171 - accuracy: 0.7396 - val_loss: 0.4555 - val_accuracy: 0.7848
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.3129 - accuracy: 0.8718 - val_loss: 0.4908 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.2058 - accuracy: 0.9193 - val_loss: 0.5468 - val_accuracy: 0.7638
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1389 - accuracy: 0.9537 - val_loss: 0.6362 - val_accuracy: 0.7769
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.1025 - accuracy: 0.9656 - val_loss: 0.6956 - val_accuracy: 0.7717


In [43]:
model_2_pred_probs = model_2.predict(val_sentence)
model_2_pred_probs[:10]

24/24 [==============================] - 1s 3ms/step


array([[0.10157805],
       [0.6796055 ],
       [0.9990219 ],
       [0.08841057],
       [0.00394124],
       [0.99901795],
       [0.69481206],
       [0.9999025 ],
       [0.9992737 ],
       [0.2212757 ]], dtype=float32)

In [44]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))

In [47]:
model_2_results = calculate_result(val_labels, model_2_preds)
model_2_results

{'accuracy': 77.16535433070865,
 'precision': 0.773049502603257,
 'recall': 0.7716535433070866,
 'f1': 0.7697096359746837}

In [52]:
import tensorflow_hub as hub
sentence_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                 input_shape=[],
                                 dtype=tf.string,
                                 trainable=False,
                                 )

In [54]:
model_3 = tf.keras.Sequential([
    sentence_layer,
    layers.Dense(1, activation="sigmoid")
])

In [55]:
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [57]:
model_3_history = model_3.fit(train_sentence,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentence, val_labels))

Epoch 1/5
215/215 [==============================] - 7s 13ms/step - loss: 0.6521 - accuracy: 0.7224 - val_loss: 0.6153 - val_accuracy: 0.7835
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.5834 - accuracy: 0.7908 - val_loss: 0.5646 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 3s 16ms/step - loss: 0.5395 - accuracy: 0.7951 - val_loss: 0.5322 - val_accuracy: 0.7913
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.5103 - accuracy: 0.7984 - val_loss: 0.5106 - val_accuracy: 0.7940
Epoch 5/5
215/215 [==============================] - 4s 17ms/step - loss: 0.4899 - accuracy: 0.8000 - val_loss: 0.4959 - val_accuracy: 0.7913


In [58]:
model_3_pred_probs = model_3.predict(val_sentence)
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))

24/24 [==============================] - 1s 15ms/step


In [62]:
model_3_results = calculate_result(val_labels, model_3_preds)
model_3_results

{'accuracy': 79.13385826771653,
 'precision': 0.7919780161707975,
 'recall': 0.7913385826771654,
 'f1': 0.790144389439456}